<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebook/PAWS_DR_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!lscpu


Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

In [2]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install --quiet -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 8102, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 8102 (delta 92), reused 123 (delta 63), pack-reused 7913
Receiving objects: 100% (8102/8102), 14.16 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (5491/5491), done.
/content/apex
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=205204 sha256=1a94ff3feb16ff19023ccc1f0ada8a7fc3d9d00f8a0be1156048590602ef9c70
  Stored in directory: /tmp/pip-ephem-wheel-cache-wc2t_3he/wheels/02/1d/54/1

In [3]:
pip install PyYAML

In [1]:
pip install -U PyYAML

In [4]:
cd ..

/content


In [5]:
!git clone -b feature/DR-images-v2 https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 336 (delta 199), reused 250 (delta 119), pack-reused 0
Receiving objects: 100% (336/336), 1.11 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [7]:
cd suncet

/content/suncet


In [8]:
pwd

'/content/suncet'

In [8]:
!mkdir datasets
!mkdir datasets/dr

In [9]:
!mkdir logs

In [11]:
!python download.py -d sample@1000

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/suncet/datasets/dr/sample@1000.zip
108MB [00:00, 111MB/s]


In [2]:
import argparse
import yaml


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [4]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [5]:
args

Namespace(devices=['cuda:0'], fname='configs/paws/dr_train.yaml', sel='paws_train')

In [ ]:
# Libraries

In [9]:
import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["SLURM_LOCALID"]
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.losses import init_paws_loss, make_labels_matrix
from src.data_manager import init_data, make_transforms, make_multicrop_transform
from src.sgd import SGD
from src.lars import LARS

import torchvision.models as models
from torch.hub import load_state_dict_from_url

import apex
from torch.nn.parallel import DistributedDataParallel

In [30]:
def load_checkpoint(r_path, encoder, opt, scaler, use_fp16=False):
    checkpoint = torch.load(r_path, map_location="cpu")
    epoch = checkpoint["epoch"]

    # -- loading encoder
    encoder.load_state_dict(checkpoint["encoder"])
    logger.info(f"loaded encoder from epoch {epoch}")

    # -- loading optimizer
    opt.load_state_dict(checkpoint["opt"])
    if use_fp16:
        scaler.load_state_dict(checkpoint["amp"])
    logger.info(f"loaded optimizers from epoch {epoch}")
    logger.info(f"read-path: {r_path}")
    del checkpoint
    return encoder, opt, epoch


def init_model(device, model_name="resnet50", use_pred=False, output_dim=128):
    if "wide_resnet" in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]() 

        # Load pre-trained ResNetImagenNet
        #logger.info("Load pre-trained ResNet ImagenNet weigths ...")
        #state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet50-0676ba61.pth',
                                              progress=True)
        #log = encoder.load_state_dict(state_dict, strict=False)
        #logger.info(log)
    
        hidden_dim = 2048
        if "w2" in model_name:
            hidden_dim *= 2
        elif "w4" in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(
        OrderedDict(
            [
                ("fc1", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn1", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu1", torch.nn.ReLU(inplace=True)),
                ("fc2", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn2", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu2", torch.nn.ReLU(inplace=True)),
                ("fc3", torch.nn.Linear(hidden_dim, output_dim)),
            ]
        )
    )

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head["bn1"] = torch.nn.BatchNorm1d(output_dim)
        pred_head["fc1"] = torch.nn.Linear(output_dim, output_dim // mx)
        pred_head["bn2"] = torch.nn.BatchNorm1d(output_dim // mx)
        pred_head["relu"] = torch.nn.ReLU(inplace=True)
        pred_head["fc2"] = torch.nn.Linear(output_dim // mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder


def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0,
):
    param_groups = [
        {
            "params": (
                p
                for n, p in encoder.named_parameters()
                if ("bias" not in n) and ("bn" not in n)
            )
        },
        {
            "params": (
                p for n, p in encoder.named_parameters() if ("bias" in n) or ("bn" in n)
            ),
            "LARS_exclude": True,
            "weight_decay": 0,
        },
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr,
    )
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup * iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs * iterations_per_epoch,
    )
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler

In [ ]:
# argparsers y demas Parameters

In [14]:
fname = args.fname
sel = args.sel

In [18]:
import pprint

In [15]:
import logging
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [19]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    params = yaml.load(y_file, Loader=yaml.FullLoader)
    logger.info('loaded params...')
    #if rank == 0:
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(params)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                's_image_folder': 'dr/sample@1000/',
                'subset_path': 'dr_subsets',
                'u_image_folder': 'dr/train_voets/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
              

In [22]:
args = params

In [26]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META
model_name = args["meta"]["model_name"]
output_dim = args["meta"]["output_dim"]
load_model = args["meta"]["load_checkpoint"]
r_file = args["meta"]["read_checkpoint"]
copy_data = args["meta"]["copy_data"]
use_fp16 = args["meta"]["use_fp16"]
use_pred_head = args["meta"]["use_pred_head"]
device = torch.device(args["meta"]["device"])
#torch.cuda.set_device(device)

In [27]:
# -- CRITERTION
reg = args["criterion"]["me_max"]
supervised_views = args["criterion"]["supervised_views"]
classes_per_batch = args["criterion"]["classes_per_batch"]
s_batch_size = args["criterion"]["supervised_imgs_per_class"]
u_batch_size = args["criterion"]["unsupervised_batch_size"]
temperature = args["criterion"]["temperature"]
sharpen = args["criterion"]["sharpen"]

In [28]:
# -- DATA
unlabeled_frac = args["data"]["unlabeled_frac"]
color_jitter = args["data"]["color_jitter_strength"]
normalize = args["data"]["normalize"]
root_path = args["data"]["root_path"]
s_image_folder = args["data"]["s_image_folder"]
u_image_folder = args["data"]["u_image_folder"]
dataset_name = args["data"]["dataset"]
subset_path = args["data"]["subset_path"]
unique_classes = args["data"]["unique_classes_per_rank"]
multicrop = args["data"]["multicrop"]
label_smoothing = args["data"]["label_smoothing"]
data_seed = None
if "cifar10" in dataset_name:
    data_seed = args["data"]["data_seed"]
    crop_scale = (0.75, 1.0) if multicrop > 0 else (0.5, 1.0)
    mc_scale = (0.3, 0.75)
    mc_size = 18
else:
    crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
    mc_scale = (0.05, 0.14)
    mc_size = 96

In [29]:
# -- OPTIMIZATION
wd = float(args["optimization"]["weight_decay"])
num_epochs = args["optimization"]["epochs"]
warmup = args["optimization"]["warmup"]
start_lr = args["optimization"]["start_lr"]
lr = args["optimization"]["lr"]
final_lr = args["optimization"]["final_lr"]
mom = args["optimization"]["momentum"]
nesterov = args["optimization"]["nesterov"]

# -- LOGGING
folder = args["logging"]["folder"]
tag = args["logging"]["write_tag"]

In [20]:
# --
log_timings = True
log_freq = 10
checkpoint_freq = 50
# --

_GLOBAL_SEED = 0
np.random.seed(_GLOBAL_SEED)
torch.manual_seed(_GLOBAL_SEED)
torch.backends.cudnn.benchmark = True

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

In [31]:
crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
mc_scale = (0.05, 0.14)
mc_size = 96

In [ ]:
# -- init torch distributed backend
#world_size, rank = init_distributed()
#logger.info(f"Initialized (rank/world-size) {rank}/{world_size}")

In [34]:
#rank=0

In [35]:
# -- log/checkpointing paths
log_file = os.path.join(folder, f"{tag}_r{rank}.csv")
save_path = os.path.join(folder, f"{tag}" + "-ep{epoch}.pth.tar")
latest_path = os.path.join(folder, f"{tag}-latest.pth.tar")
best_path = os.path.join(folder, f"{tag}" + "-best.pth.tar")
load_path = None
if load_model:
    load_path = os.path.join(folder, r_file) if r_file is not None else latest_path


In [36]:
# -- make csv_logger
csv_logger = CSVLogger(
    log_file,
    ("%d", "epoch"),
    ("%d", "itr"),
    ("%.5f", "paws-xent-loss"),
    ("%.5f", "paws-me_max-reg"),
    ("%d", "time (ms)"),
)

In [ ]:
# -- init model
encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim,
)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


In [ ]:
if world_size > 1:
    process_group = apex.parallel.create_syncbn_process_group(0)
    encoder = apex.parallel.convert_syncbn_model(
        encoder, process_group=process_group
    )

In [ ]:
# -- init losses
paws = init_paws_loss(multicrop=multicrop, tau=temperature, T=sharpen, me_max=reg)

In [ ]:
# -- assume support images are sampled with ClassStratifiedSampler
labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=world_size,
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing,
)

In [ ]:
# -- make data transforms
transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize,
)
multicrop_transform = (multicrop, None)
if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
        dataset_name=dataset_name,
        num_crops=multicrop,
        size=mc_size,
        crop_scale=mc_scale,
        normalize=normalize,
        color_distortion=color_jitter,
    )

In [ ]:
# -- init data-loaders/samplers
(
    unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler,
) = init_data(
    dataset_name=dataset_name,
    transform=transform,
    init_transform=init_transform,
    supervised_views=supervised_views,
    u_batch_size=u_batch_size,
    s_batch_size=s_batch_size,
    unique_classes=unique_classes,
    classes_per_batch=classes_per_batch,
    multicrop_transform=multicrop_transform,
    world_size=world_size,
    rank=rank,
    root_path=root_path,
    s_image_folder=s_image_folder,
    u_image_folder=u_image_folder,
    training=True,
    copy_data=copy_data,
)
iter_supervised = None
ipe = len(unsupervised_loader)
logger.info(f"iterations per epoch: {ipe}")

In [ ]:
# -- init optimizer and scheduler
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
    encoder=encoder,
    weight_decay=wd,
    start_lr=start_lr,
    ref_lr=lr,
    final_lr=final_lr,
    ref_mom=mom,
    nesterov=nesterov,
    iterations_per_epoch=ipe,
    warmup=warmup,
    num_epochs=num_epochs,
)
if world_size > 1:
    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)

In [ ]:
start_epoch = 0
# -- load training checkpoint
if load_model:
    encoder, optimizer, start_epoch = load_checkpoint(
        r_path=load_path,
        encoder=encoder,
        opt=optimizer,
        scaler=scaler,
        use_fp16=use_fp16,
    )
    for _ in range(start_epoch):
        for _ in range(ipe):
            scheduler.step()

In [ ]:
# -- TRAINING LOOP
best_loss = None
for epoch in range(start_epoch, num_epochs):
    logger.info("Epoch %d" % (epoch + 1))

    # -- update distributed-data-loader epoch
    unsupervised_sampler.set_epoch(epoch)
    if supervised_sampler is not None:
        supervised_sampler.set_epoch(epoch)

    loss_meter = AverageMeter()
    ploss_meter = AverageMeter()
    rloss_meter = AverageMeter()
    time_meter = AverageMeter()
    data_meter = AverageMeter()

    for itr, udata in enumerate(unsupervised_loader):

        def load_imgs():
            # -- unsupervised imgs
            uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
            # -- supervised imgs
            global iter_supervised
            try:
                sdata = next(iter_supervised)
            except Exception:
                iter_supervised = iter(supervised_loader)
                logger.info(f"len.supervised_loader: {len(iter_supervised)}")
                sdata = next(iter_supervised)
            finally:
                labels = torch.cat([labels_matrix for _ in range(supervised_views)])
                simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
            # -- concatenate supervised imgs and unsupervised imgs
            imgs = simgs + uimgs
            return imgs, labels

        (imgs, labels), dtime = gpu_timer(load_imgs)
        data_meter.update(dtime)

        def train_step():
            with torch.cuda.amp.autocast(enabled=use_fp16):
                optimizer.zero_grad()

                # --
                # h: representations of 'imgs' before head
                # z: representations of 'imgs' after head
                # -- If use_pred_head=False, then encoder.pred (prediction
                #    head) is None, and _forward_head just returns the
                #    identity, z=h
                h, z = encoder(imgs, return_before_head=True)

                # Compute paws loss in full precision
                with torch.cuda.amp.autocast(enabled=False):

                    # Step 1. convert representations to fp32
                    h, z = h.float(), z.float()

                    # Step 2. determine anchor views/supports and their
                    #         corresponding target views/supports
                    # --
                    num_support = (
                        supervised_views * s_batch_size * classes_per_batch
                    )
                    # --
                    anchor_supports = z[:num_support]
                    anchor_views = z[num_support:]
                    # --
                    target_supports = h[:num_support].detach()
                    target_views = h[num_support:].detach()
                    target_views = torch.cat(
                        [
                            target_views[u_batch_size : 2 * u_batch_size],
                            target_views[:u_batch_size],
                        ],
                        dim=0,
                    )

                    # Step 3. compute paws loss with me-max regularization
                    (ploss, me_max) = paws(
                        anchor_views=anchor_views,
                        anchor_supports=anchor_supports,
                        anchor_support_labels=labels,
                        target_views=target_views,
                        target_supports=target_supports,
                        target_support_labels=labels,
                    )
                    loss = ploss + me_max

            scaler.scale(loss).backward()
            lr_stats = scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            return (float(loss), float(ploss), float(me_max), lr_stats)

        (loss, ploss, rloss, lr_stats), etime = gpu_timer(train_step)
        loss_meter.update(loss)
        ploss_meter.update(ploss)
        rloss_meter.update(rloss)
        time_meter.update(etime)

        if (itr % log_freq == 0) or np.isnan(loss) or np.isinf(loss):
            csv_logger.log(
                epoch + 1, itr, ploss_meter.avg, rloss_meter.avg, time_meter.avg
            )
            logger.info(
                "[%d, %5d] loss: %.5f (%.5f %.5f) "
                "(%d ms; %d ms)"
                % (
                    epoch + 1,
                    itr,
                    loss_meter.avg,
                    ploss_meter.avg,
                    rloss_meter.avg,
                    time_meter.avg,
                    data_meter.avg,
                )
            )
            if lr_stats is not None:
                logger.info(
                    "[%d, %5d] lr_stats: %.5f (%.2e, %.2e)"
                    % (epoch + 1, itr, lr_stats.avg, lr_stats.min, lr_stats.max)
                )

        assert not np.isnan(loss), "loss is nan"

    # -- logging/checkpointing
    logger.info("avg. loss %.3f" % loss_meter.avg)

    if rank == 0:
        save_dict = {
            "encoder": encoder.state_dict(),
            "opt": optimizer.state_dict(),
            "epoch": epoch + 1,
            "unlabel_prob": unlabeled_frac,
            "loss": loss_meter.avg,
            "s_batch_size": s_batch_size,
            "u_batch_size": u_batch_size,
            "world_size": world_size,
            "lr": lr,
            "temperature": temperature,
            "amp": scaler.state_dict(),
        }
        torch.save(save_dict, latest_path)
        if best_loss is None or best_loss > loss_meter.avg:
            best_loss = loss_meter.avg
            logger.info('updating "best" checkpoint')
            torch.save(save_dict, best_path)
        if (
            (epoch + 1) % checkpoint_freq == 0
            or (epoch + 1) % 10 == 0
            and epoch < checkpoint_freq
        ):
            torch.save(save_dict, save_path.format(epoch=f"{epoch + 1}"))

In [ ]:
#Loss functions

In [ ]:
def train_resnet():
    """
    Aqui tengo que incluir los dataloaders y demas

    definir el train_loop_fn
    """

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
    global FLAGS
    FLAGS = flags
    torch.set_default_tensor_type("torch.FloatTensor")
    accuracy, data, pred, target = train_resnet()


In [ ]:
# TO DO 

if __name__ == "__main__":
    parser = init_argparse()
    FLAGS = parser.parse_args()
    print(FLAGS)
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS.num_cores, start_method="fork")